# Downloading and preparing the data

In [10]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import LearningRateScheduler
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

In [11]:
(train_x, train_y), (test_x, test_y) = datasets.cifar10.load_data()

In [12]:
train_x, test_x = train_x / 255.0, test_x / 255.0 #changes pixel values between 0 and 1

# Improving the Basic Model

In [33]:
model = models.Sequential()
model.add(layers.Conv2D(64, (3,3), padding='same', activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.0001), input_shape=(32,32,3)))#creates layers for a 2d array of pixels
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(64,(3,3), padding='same', kernel_regularizer=tf.keras.regularizers.l2(0.0001), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2,2))) #pooling helps detect features in an image, max pooling finds largest value
model.add(layers.Dropout(0.3))

model.add(layers.Conv2D(128,(3,3), padding='same', kernel_regularizer=tf.keras.regularizers.l2(0.0001), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(128,(3,3), padding='same', kernel_regularizer=tf.keras.regularizers.l2(0.0001), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Dropout(0.3))

model.add(layers.Conv2D(256,(3,3), padding='same', kernel_regularizer=tf.keras.regularizers.l2(0.0001), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(256,(3,3), padding='same', kernel_regularizer=tf.keras.regularizers.l2(0.0001), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Dropout(0.3))

model.add(layers.Flatten())
model.add(layers.Dense(128, activation = 'relu'))#regularisation of data
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation = 'softmax'))
model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_66 (Conv2D)           (None, 32, 32, 64)        1792      
_________________________________________________________________
batch_normalization_49 (Batc (None, 32, 32, 64)        256       
_________________________________________________________________
conv2d_67 (Conv2D)           (None, 32, 32, 64)        36928     
_________________________________________________________________
batch_normalization_50 (Batc (None, 32, 32, 64)        256       
_________________________________________________________________
max_pooling2d_33 (MaxPooling (None, 16, 16, 64)        0         
_________________________________________________________________
dropout_35 (Dropout)         (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_68 (Conv2D)           (None, 16, 16, 128)     



# Running, training, and evaluating data

In [34]:
model.compile(optimizer='RMSprop',loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])

datagen = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)

batch = 512
#create iterator - training data
it_train = datagen.flow(train_x, tf.squeeze(train_y), batch_size = batch) #batch_size default= 32

early_stopping = EarlyStopping(monitor='val_accuracy', patience=20, restore_best_weights=True)

def scheduler(epoch, lr):
  if epoch < 70:
    return lr
  if epoch < 90:
    return lr * tf.math.exp(-0.3)
  else:
    return lr * tf.math.exp(-0.1)


learning_rate_scheduling = LearningRateScheduler(scheduler)

progress = model.fit(it_train, epochs=200, batch_size=batch, shuffle=True, callbacks=[early_stopping, learning_rate_scheduling], validation_data=(test_x, test_y))

Epoch 1/200
98/98 [==============================] - 25s 238ms/step - loss: 2.3788 - accuracy: 0.2971 - val_loss: 3.1614 - val_accuracy: 0.1000
Epoch 2/200
98/98 [==============================] - 23s 235ms/step - loss: 1.4301 - accuracy: 0.5339 - val_loss: 2.6621 - val_accuracy: 0.1411
Epoch 3/200
98/98 [==============================] - 23s 235ms/step - loss: 1.1015 - accuracy: 0.6515 - val_loss: 2.5684 - val_accuracy: 0.1882
Epoch 4/200
98/98 [==============================] - 23s 236ms/step - loss: 0.9256 - accuracy: 0.7105 - val_loss: 2.1277 - val_accuracy: 0.3509
Epoch 5/200
98/98 [==============================] - 23s 233ms/step - loss: 0.8191 - accuracy: 0.7512 - val_loss: 1.3486 - val_accuracy: 0.5689
Epoch 6/200
98/98 [==============================] - 23s 235ms/step - loss: 0.7604 - accuracy: 0.7751 - val_loss: 0.9477 - val_accuracy: 0.7094
Epoch 7/200
98/98 [==============================] - 23s 237ms/step - loss: 0.7155 - accuracy: 0.7902 - val_loss: 0.7745 - val_accuracy:

In [35]:
loss, acc = model.evaluate(test_x, test_y, verbose = 2)
print(acc)

313/313 - 1s - loss: 0.4605 - accuracy: 0.9221
0.9221000075340271
